## Lisamaterjal: Märgenduste visualiseerimine II

Selles lisamaterjalis tutvustame mõningaid keerukamaid märgenduste visualiseerimise viise, mis nõuavad teadmisi ka objekt-orienteeritud programmeerimisest.

### Klass `DisplaySpans`

Meeldetuletuseks: lihtne viis märgendusi visualiseerida / tekstis esile tuua on kihi meetodi `display()` abil:

In [1]:
# Loome näidisteksti
from estnltk import Text
text = Text('Ooperitäht José Carreras saabus eralennukiga Tallinna.').tag_layer()

# Lisame tekstile nimeüksuste märgenduse
from estnltk.taggers import NerTagger
ner_tagger = NerTagger()
ner_tagger.tag(text)
# Kuvame kõik nimeüksused värviliselt
text.ner.display()

Ooperitäht José Carreras saabus eralennukiga Tallinna .

Kui kutsume välja kihi meetodi `display()`, siis taustal toimub tegelikult `DisplaySpans` objekti loomine ning kihi etteandmine objektile, misjärel toimub kihi kuvamine:

In [2]:
from estnltk.visualisation import DisplaySpans

display_spans = DisplaySpans( mapping_dict={"font-weight": lambda x: 'bold'} )
display_spans( text.ner )

Ooperitäht José Carreras saabus eralennukiga Tallinna .

Kui on vaja kätte saada kuvatud HTML lähtekoodi (nt faili salvestamiseks), siis selle saab kätte meetodi `html_output()` abil:

In [3]:
display_spans.html_output( text.ner  )

'<script>\nvar elements = document.getElementsByClassName("overlapping-span")\nfor (let i = 0; i < elements.length; i++){\n    elements.item(i).addEventListener("click",function() {show_conflicting_spans(elements.item(i));})}\n\nfunction show_conflicting_spans(span_element) {\n    let spantable = document.createElement(\'div\')\n    spantable.classList.add(\'tables\')\n\n    // Prepare the contents of the span table\n    data = span_element.getAttribute("span_info")\n    data = data.split(",")\n    var spancontent = \'<table>\'\n    for (let row of data) {\n        spancontent+=\'<tr><td>\'\n        spancontent+=row\n        spancontent+=\'</td></tr>\'\n    }\n    spancontent += \'</table>\'\n    spantable.innerHTML = spancontent\n    span_element.parentElement.appendChild(spantable)\n\n    // Increase the size of the cell so the tables would fit\n    spantable.parentElement.style.height = Math.max(Number(spantable.parentElement.style.height.substring(0,spantable.parentElement.style.he

---

### Keerukamad visualiseerimisvõimalused: klassi `DisplaySpans` üledefineerimine

Juhendmaterjalis https://nbviewer.jupyter.org/github/estnltk/estnltk/blob/version_1.6/tutorials/visualisation/span_visualisation.ipynb on tutvustatud programmatiliselt ka veidi keerukamaid visualiseerimisvõimalusi: mitmese morf analüüsi saanud sõnade esiletoomine, eri sõnaliiki kuuluvate sõnade esiletoomine, liitsõnede ( _compound tokens_ ) esiletoomine vastavalt tüübile ja käänete ning liitsõnade esiletoomine. 
Põhiline idee on kõigis näidetes sarnane: klassi `DisplaySpans` üle defineerides luuakse uus klass, kus on defineeritud konstruktor ja  meetodid:
   * `__call` ~~ kuvamise suunamine vastavalt sellele, kas sisendiks on `Text` või `Layer`;
   * `restore_defaults` ~~ kuvamise vaikesätete loomine;
   * `__default_ambiguity_resolver` ~~ mitmesuste lahendamine enne taustavärvi määramist;
   * `__bg_mapper` ~~ taustavärvi määramine etteantud tekstiüksusele;



Vaatame sealsetest näidetest lähemalt sõnaliikide näidet ( `DisplayPostagsSpans` ).

Kõigepealt tuleb luua klass `DirectPlainSpanVisualiser`, mida rakendatakse `DisplayPostagsSpans` poolt igal potentsiaalselt märgendataval tekstijupil.
Klassis on CSS elementide kaupa määratud funktsioonid, mis saavad sisendiks jooksva tekstijupi ja sellele vastavad märgendused ning tagastavad väärtuse, mis tuleb CSS elemendile määrata:

In [4]:
from estnltk.visualisation.core.span_visualiser import SpanVisualiser
import html

class DirectPlainSpanVisualiser(SpanVisualiser):
    """Class that visualises spans, arguments can be css elements.
    Arguments that can be changed are bg_mapping, colour_mapping, font_mapping, weight_mapping,
    italics_mapping, underline_mapping, size_mapping and tracking_mapping. These should
    be functions that take the span as the argument and return a string that will be
    the value of the corresponding attribute in the css."""

    def __init__(self, colour_mapping=None, bg_mapping=None, font_mapping=None,
                 weight_mapping=None, italics_mapping=None, underline_mapping=None,
                 size_mapping=None, tracking_mapping=None, fill_empty_spans=False):

        self.bg_mapping = bg_mapping or self.default_bg_mapping
        self.colour_mapping = colour_mapping
        self.font_mapping = font_mapping
        self.weight_mapping = weight_mapping
        self.italics_mapping = italics_mapping
        self.underline_mapping = underline_mapping
        self.size_mapping = size_mapping
        self.tracking_mapping = tracking_mapping
        self.fill_empty_spans = fill_empty_spans

    def __call__(self, segment, spans):

        segment[0] = html.escape(segment[0])

        # Simple text no span to fill
        if not self.fill_empty_spans and self.is_pure_text(segment):
            return segment[0]

        # There is a span to decorate
        output = ['<span style=']
        if self.colour_mapping is not None:
            output.append('color:' + self.colour_mapping(segment, spans) + ";")
        if self.bg_mapping is not None:
            output.append('background:' + self.bg_mapping(segment, spans) + ";")
        if self.font_mapping is not None:
            output.append('font-family:' + self.font_mapping(segment, spans) + ";")
        if self.weight_mapping is not None:
            output.append('font-weight:' + self.weight_mapping(segment, spans) + ";")
        if self.italics_mapping is not None:
            output.append('font-style:' + self.italics_mapping(segment, spans) + ";")
        if self.underline_mapping is not None:
            output.append('text-decoration:' + self.underline_mapping(segment, spans) + ";")
        if self.size_mapping is not None:
            output.append('font-size:' + self.size_mapping(segment, spans) + ";")
        if self.tracking_mapping is not None:
            output.append('letter-spacing:' + self.tracking_mapping(segment, spans) + ";")
        if len(segment[1]) > 1:
            output.append(' class=overlapping-span ')
            rows = []
            for i in segment[1]:
                rows.append(spans[i].text)
            output.append(' span_info=' + html.escape(','.join(rows)))  # text of spans for javascript
        output.append('>')
        output.append(segment[0])
        output.append('</span>')
        return "".join(output)

Edasi loome `DisplayPostagsSpans` klassi, mis tegeleb sõnade värvimisega vastavalt nende kuulumisele eri sõnaliikidesse:

In [5]:
from collections import defaultdict
from typing import Mapping, Any, Tuple, List, Sequence, Union

from estnltk import Text, Layer
from estnltk.visualisation import DisplaySpans

class DisplayPostagsSpans(DisplaySpans):
    """
    Visualises different part-of-speech tags in a text
    
    Provides default background colourschme for EstMorf and GT tagsets.
    Color scheme is controlled by two dictionary-like class attributes
    * pos_coloring[str]
    * span_coloring[int]
    
    The first coloring controls how spans with different POS-tags are 
    colored. Default coloring can be changed by assigning appropriate
    entries, e.g. pos_coloring['V'] = 'black'.
    
    The second controls how span overlaps are colored. The tokenization 
    into the words can be ambiguous. By default, overlaps are colored
    by two shades of red. This can be changed by assigning appropriate
    entries, e.g. span_coloring[2] = 'blue'.
    
    To redefine the entire color scheme, the entire colouring attribute
    must be redefined. The assigned object must support indexing with 
    any string for pos_coloring and any int for span_coloring.
    
    As POS-tagging may be ambiguous, coloring is done in two phases:
    1. list of POS-tags is aggregated into a new string label
    2. POS-tag coloring is used to determine the background color
    
    The default aggregator marks all ambigious labellings with '*'.
    It is possible to customise this by redefining ambiguity_resolver.
    """

    def __init__(self, layer:str='morph_analysis', tagset:str='EstMorf', ambiguity_resolver:callable=None):
        #super(DisplayPostagsSpans, self).__init__(styling="direct")
        super(DisplayPostagsSpans, self).__init__()
        
        # Hack to get it working by replacing a wrong base class
        self.span_decorator = DirectPlainSpanVisualiser()

        self.morph_layer = layer
        self.tagset = tagset
        self.__default_ambiguity_resolver = ambiguity_resolver or self.__default_ambiguity_resolver
        self.span_decorator.bg_mapping = self.__bg_mapper
        self.restore_defaults()
        
        
    def restore_defaults(self): 
        """Restore default coloring scheme for part-of-speech tags and token overlaps and ambiguity resolver"""
        
        self.ambiguity_resolver = self.__default_ambiguity_resolver
        
        self.pos_coloring = {}
        if self.tagset == 'EstMorf' or self.tagset == 'GT':
            self.pos_coloring['S'] = 'orange'
            self.pos_coloring['H'] = 'orange'
            self.pos_coloring['A'] = 'yellow'
            self.pos_coloring['U'] = 'yellow'
            self.pos_coloring['C'] = 'yellow'
            self.pos_coloring['N'] = 'yellow'
            self.pos_coloring['O'] = 'yellow'
            self.pos_coloring['V'] = 'lime'
            self.pos_coloring['*'] = 'gray'
            
        # Define two shades of red for overlapping tokenization
        self.span_coloring = {2:'#FF5050'}
        
            
    def __call__(self, object:Union[Text, Layer]) -> str:
        if isinstance(object, Text):
            return super(DisplayPostagsSpans, self).__call__(object[self.morph_layer])
        elif isinstance(object, Layer):
            return super(DisplayPostagsSpans, self).__call__(object)
        else:
            raise ValueError('Invalid input')
            
            
    def __default_ambiguity_resolver(self, span) -> str:
        pos_tags = set(span['partofspeech'])
        if len(pos_tags) == 1:
            return next(iter(pos_tags));
        return '*'

    
    def __bg_mapper(self, segment: Tuple[str, List[int]], spans) -> str:
        
        if len(segment[1]) != 1:
            return self.span_coloring.get(len(segment[1]),'#FF0000')
            
        return self.pos_coloring.get(self.ambiguity_resolver(spans[segment[1][0]]),'#ffffff00')

Initsialiseerime `DisplayPostagsSpans` klassi ning muudame vaikeväärtuseid selliselt, et sõnaliikide värvid oleks üksteisest paremini eristatud:

In [6]:
# Määra sõnaliikidele värvid
display_postags = DisplayPostagsSpans()
# verbid
display_postags.pos_coloring['V'] = '#037ffc' # sinine
# nimisõnad
display_postags.pos_coloring['S'] = '#f5af9a' # punakas
display_postags.pos_coloring['H'] = '#f28563' # (tumedam) punane
display_postags.pos_coloring['Y'] = '#d65f5f' # (tumedam) punane
# omadussõnad
display_postags.pos_coloring['A'] = 'yellow'   # kollane
display_postags.pos_coloring['C'] = '#f7d034'  # (tumedam) kollane
display_postags.pos_coloring['U'] = '#d9b321'  # (tumedam) kollane
# arvsõnad
display_postags.pos_coloring['N'] = '#40e68d'  # roheline
display_postags.pos_coloring['O'] = '#5fba7d'  # (tumedam) roheline

Loome näidisteksti ning visualiseerime selles sõnaliigid:

In [7]:
# Näidistekst
# Allikas: https://et.wikipedia.org/wiki/Pruunkaru
text = Text('''
Pruunkaru on suurim Eestis elav kiskjaline ja suurim Euroopa mandriosas elav kiskjaline. 
Toitumiselt on karu kõigesööja ja valdava osa toidust moodustavad mitmesugused marjad, seened, 
seemned ja putukad.

Küttimise ja elupaikade hävitamise tõttu on pruunkaru levila ahenenud. Pruunkarude koguarv 
maailmas on hinnanguliselt 185 000 – 200 000.
''').tag_layer()

# Tekst värviliste sõnaliikidega
display_postags(text.morph_analysis)

Pruunkaru on suurim Eestis elav kiskjaline ja suurim Euroopa mandriosas elav kiskjaline . Toitumiselt on karu kõigesööja ja valdava osa toidust moodustavad mitmesugused marjad , seened , seemned ja putukad . Küttimise ja elupaikade hävitamise tõttu on pruunkaru levila ahenenud . Pruunkarude koguarv maailmas on hinnanguliselt 185 000 – 200 000 .

Teadmisi HTML-i ja sõnaliigitähiste kohta appi võttes saame luua ka funktsiooni, mis tekitab meile nn värvide legendi:

In [8]:
import pandas
from IPython.display import HTML, display
    
def legend(display_postags):
    table = [ '<h4>Legend</h4>' ]
    pos_listing = { pos: display_postags.pos_coloring[pos] for pos in display_postags.pos_coloring }
    name_map = { '*': 'mitmene', 
                 'S': 'nimisõna', 
                 'H': 'pärisnimi', 
                 'Y': 'lühend', 
                 'V': 'verb', 
                 'A': 'omadussõna', 
                 'C': 'omadussõna (keskvõrre)', 
                 'U': 'omadussõna (ülivõrre)', 
                 'N': 'põhiarvsõna', 
                 'O': 'järgarvsõna', }
    table.append('<table>')
    table.append('<tr>')
    for pos in ['S', 'H', 'Y', 'P', 'N', 'O', 'V', 'A', 'C', 'U', '*']:
        if pos in pos_listing:
            table.append('<td style="background-color: '+pos_listing[pos]+';">')
            table.append( '<b>' )
            table.append( name_map.get(pos, pos) )
            table.append( '</b>' )
            table.append('</td>')

    table.append('</tr>')
    table.append('</table>')
    return display(HTML('\n'.join(table)))

In [9]:
# Tekst värviliste sõnaliikidega
display_postags(text.morph_analysis)
# ning värvide legend
legend(display_postags)

Pruunkaru on suurim Eestis elav kiskjaline ja suurim Euroopa mandriosas elav kiskjaline . Toitumiselt on karu kõigesööja ja valdava osa toidust moodustavad mitmesugused marjad , seened , seemned ja putukad . Küttimise ja elupaikade hävitamise tõttu on pruunkaru levila ahenenud . Pruunkarude koguarv maailmas on hinnanguliselt 185 000 – 200 000 .

nimisõna,pärisnimi,lühend,põhiarvsõna,järgarvsõna,verb,omadussõna,omadussõna (keskvõrre),omadussõna (ülivõrre),mitmene


---